In [0]:
DEBUG = False
SECTOR = 'all'
FIELD = 'm31'
OBS_FILE = f'$PFS_TARGETING_DATA/data/targeting/m31/M31Catalog_forPFS.csv'
PROJECT_DIR = '/home/dobos/project'
CONFIG_FILES = [
    f'{PROJECT_DIR}/ga_targeting/configs/netflow/SSP/m31/_common.py',
    f'{PROJECT_DIR}/ga_targeting/configs/netflow/SSP/m31/{FIELD}.py'
]
ASSIGNMENTS_FILE = ''

# Observation time frame in HST
TIME_FROM = '2025-09-13 12:00:00'
TIME_TO = '2025-09-15 12:00:00'

In [0]:
import os, sys
from datetime import datetime
import dateutil.parser
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Ellipse, Circle
from matplotlib.gridspec import GridSpec

In [0]:
plt.rc('font', size=6)

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
if DEBUG and 'debug' not in globals():
    import debugpy
    debugpy.listen(('0.0.0.0', 5698))
    debug = True

# Plot definitions

In [0]:
from astropy.time import Time, TimeDelta
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_body
from astroplan import Observer
from astroplan.moon import moon_phase_angle, moon_illumination
import astropy.units as u

from pfs.utils.coordinates import Subaru_POPT2_PFS

from pfs.ga.targeting.targets.m31 import M31_SECTORS #, M31_FIELDS
from pfs.ga.targeting.instrument import *
from pfs.ga.common.diagram import CMD, CCD, FOV, FP, ColorAxis, MagnitudeAxis
from pfs.ga.common.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import Pointing, WcsProjection
from pfs.ga.targeting.config.netflow import NetflowConfig

In [0]:
field = M31_SECTORS[SECTOR]
hsc = field.get_photometry()
cmd = field.get_cmd()
ccd = field.get_ccd()

In [0]:
pointing = field.get_center()
pointing

In [0]:
wcs = WcsProjection(pointing, proj='TAN')
fov = FOV(projection=wcs)

# Load the config files

In [0]:
config = field.get_netflow_config()
config.load(CONFIG_FILES, ignore_collisions=True)

In [0]:
config.pointings

In [0]:
config.instrument_options

# Load observations

In [0]:
from pfs.ga.targeting.io import ObservationSerializer

In [0]:
obs = field.get_text_observation_reader().read(os.path.expandvars(OBS_FILE))
obs.data.shape

# Calculate the whitening matrix

This basically determines the second moments of the coordinate distribution
and yield the axes of the ellipsoid and the position angle

In [0]:
from pfs.ga.targeting.projection import Whitening
from pfs.ga.targeting.radialprofile import King

In [0]:
# Iteratively determine a whitening matrix that converts the
# spherical coordinates into deprojected coordinates

w = Whitening(projection=wcs)

if FIELD == 'm31':
    w_mask = w.create(obs, iterations=30, s_cut=2.2)
else:
    raise NotImplementedError()

w.M, w.reverse(w.M), field.pos

In [0]:
[dsph_maj, dsph_min] = w.S
dsph_pa = np.degrees(np.angle(w.Vh[0, 0] + 1j * w.Vh[1, 0]))
dsph_maj, dsph_min, dsph_pa

In [0]:
f, ax = plt.subplots(1, 1, figsize=(8, 8), dpi=240)

(wx, wy) = w.apply(obs.get_coords())
ax.plot(wx, wy, '.', ms=0.1)
ax.plot(wx[w_mask], wy[w_mask], '.', ms=0.1)
ax.set_aspect('equal')
ax.grid()

In [0]:
w.M, w.W, w.reverse(w.M)

In [0]:
w.M, w.reverse(w.M), field.pos

# Define pointings

In [0]:
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy.coordinates import CartesianRepresentation
from astropy.coordinates.matrix_utilities import rotation_matrix
import astropy.units as u

from pfs.ga.targeting.projection import Pointing

In [0]:
# Cartesian coordinate system
SkyCoord(0, 90, unit='deg').cartesian       # (0, 0, 1)     -> z axis points to north
SkyCoord(0, 0, unit='deg').cartesian         # (1, 0, 0)     -> x axis points to us
SkyCoord(90, 0, unit='deg').cartesian        # (0, 1, 0)     -> y axis points to east

In [0]:
def get_full_rotation(sep, pa, ra, dec):
    r1 = rotation_matrix(sep * u.deg, 'z')  # separation of the pointing center from the center of the object
    r2 = rotation_matrix(pa * u.deg, 'x')   # position angle of the ellipse
    r3 = rotation_matrix(dec * u.deg, 'y')    # declination
    r4 = rotation_matrix(-ra * u.deg, 'z')    # right ascension

    return r4 @ r3 @ r2 @ r1

def calculate_center(sep, pa, ra, dec):
    r = get_full_rotation(sep, pa, ra, dec)
    c = SkyCoord(CartesianRepresentation(1, 0, 0).transform(r))
    return c.ra.value, c.dec.value

# print(r1, r2, r3, r4, r4 @ r3 @ r2 @ r1)

# print(SkyCoord(CartesianRepresentation(1, 0, 0)))
# print(SkyCoord(CartesianRepresentation(1, 0, 0).transform(r1)))
# print(SkyCoord(CartesianRepresentation(1, 0, 0).transform(r2 @ r1)))
# print(SkyCoord(CartesianRepresentation(1, 0, 0).transform(r3 @ r2 @ r1)))
# print(SkyCoord(CartesianRepresentation(1, 0, 0).transform(r4 @ r3 @ r2 @ r1)))

In [0]:
field.pos.ra.value, field.pos.dec.value, calculate_center(1.5, 40, field.pos.ra.value, field.pos.dec.value)

In [0]:
center = field.get_center()
obs_time = Time('2025-09-13T10:00:00')        # midnight in Hawaii

# Use the field defaults
pointings = field.get_pointings(SubaruPFI)

# Use the config file
# pointings = [ p.get_pointing(obs_time=obs_time) for p in config.pointings ]

center, pointings

In [0]:
field

In [0]:
# Generate pointings in a pattern
# Use constants for the center of M31 in decimal degrees

def hex(u,v):
    r = np.sqrt(u**2 + u*v + v**2)
    theta1 = np.rad2deg(np.arctan2(np.sqrt(3)*v, 2*u + v)) % 360
    theta2 = np.rad2deg(np.arctan2(np.sqrt(3)*u, 2*v + u)) % 360
    theta3 = np.rad2deg(np.arctan2(-np.sqrt(3)*v, -2*u - v)) % 360
    theta4 = np.rad2deg(np.arctan2(-np.sqrt(3)*u, -2*v - u)) % 360
    theta = [theta1, theta2, theta3, theta4]
    theta = np.unique(theta)
    theta = np.sort(theta)
    return r, theta

print(hex(4, 1))


In [0]:

SEP = 1.15

decE = 41.2687 - 0.1*SEP
raE = 10.6847 + 0.35*SEP/np.cos(np.deg2rad(decE))
paE = -20
fieldpaE = (70 - paE) % 360
decW = 41.2687 + 0.3*SEP
raW = 10.6847 - 0.45*SEP/np.cos(np.deg2rad(decW))
paW = -25
fieldpaW = (70 - paW) % 360
pointings = [
    #Pointing(raE, decE, posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0),

    #Pointing(*calculate_center(SEP, paE+0, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(SEP, paE+60, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(SEP, paE+120, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0),
    Pointing(*calculate_center(SEP, paE+180, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0, label='diskE_2'),
    Pointing(*calculate_center(SEP, paE+240, raE-0.25*SEP/np.cos(np.deg2rad(decE))*np.sin(np.deg2rad(paE)), decE-0.0*SEP*np.cos(np.deg2rad(paE))), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0, label='diskE_3'),
    Pointing(*calculate_center(SEP, paE+300, raE-0.3*SEP/np.cos(np.deg2rad(decE))*np.sin(np.deg2rad(paE)), decE-0.3*SEP*np.cos(np.deg2rad(paE))), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0, label='GSS_1'),

    #Pointing(*calculate_center(2 * SEP, paE+0, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(2 * SEP, paE+60, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(2 * SEP, paE+120, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0),
    Pointing(*calculate_center(2 * SEP, paE+180, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0, label='haloE_1'),
    Pointing(*calculate_center(2 * SEP, paE+240, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0, label='haloE_3'),
    Pointing(*calculate_center(2 * SEP, paE+300, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0, label='GSS_4'),

    #Pointing(*calculate_center(np.sqrt(3) * SEP, paE+30, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(np.sqrt(3) * SEP, paE+90, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0),
    Pointing(*calculate_center(np.sqrt(3) * SEP, paE+150, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0, label='diskE_1'),
    Pointing(*calculate_center(np.sqrt(3) * SEP, paE+210, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0, label='haloE_2'),
    Pointing(*calculate_center(np.sqrt(3) * SEP, paE+270, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0, label='GSS_3'),
    Pointing(*calculate_center(np.sqrt(3) * SEP, paE+330, raE-0.35*SEP/np.cos(np.deg2rad(decE))*np.sin(np.deg2rad(paE)), decE-0.35*SEP*np.cos(np.deg2rad(paE))), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0, label='GSS_2'),

    #Pointing(*calculate_center(3 * SEP, paE+0, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(3 * SEP, paE+60, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(3 * SEP, paE+120, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(3 * SEP, paE+180, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0),
    Pointing(*calculate_center(3 * SEP, paE+240, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0, label='haloE_6'),
    #Pointing(*calculate_center(3 * SEP, paE+300, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0),

    #Pointing(*calculate_center(np.sqrt(7) * SEP, paE+20, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(np.sqrt(7) * SEP, paE+40, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(np.sqrt(7) * SEP, paE+80, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(np.sqrt(7) * SEP, paE+100, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(np.sqrt(7) * SEP, paE+140, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(np.sqrt(7) * SEP, paE+160, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0),
    Pointing(*calculate_center(np.sqrt(7) * SEP, paE+200, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0, label='haloE_4'),
    Pointing(*calculate_center(np.sqrt(7) * SEP, paE+220, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0, label='haloE_5'),
    Pointing(*calculate_center(np.sqrt(7) * SEP, paE+260, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0, label='GSS_5'),
    Pointing(*calculate_center(np.sqrt(7) * SEP, paE+280, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0, label='GSS_6'),
    #Pointing(*calculate_center(np.sqrt(7) * SEP, paE+320, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(np.sqrt(7) * SEP, paE+340, raE, decE), posang=fieldpaE-paE, obs_time=obs_time, stage=0, priority=0),

    Pointing(*calculate_center(SEP, paW+0, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0, label='diskW_4'),
    Pointing(*calculate_center(SEP, paW+60, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0, label='diskW_3'),
    Pointing(*calculate_center(SEP, paW+120, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0, label='diskW_2'),
    #Pointing(*calculate_center(SEP, paW+180, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(SEP, paW+240, raW-0.25*SEP/np.cos(np.deg2rad(decW))*np.sin(np.deg2rad(paW)), decW-0.0*SEP*np.cos(np.deg2rad(paW))), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(SEP, paW+300, raW-0.3*SEP/np.cos(np.deg2rad(decW))*np.sin(np.deg2rad(paW)), decW-0.3*SEP*np.cos(np.deg2rad(paW))), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),

    #Pointing(*calculate_center(2 * SEP, paW+0, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),
    Pointing(*calculate_center(2 * SEP, paW+60, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0, label='haloW_6'),
    Pointing(*calculate_center(2 * SEP, paW+120, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0, label='diskW_1'),
    #Pointing(*calculate_center(2 * SEP, paW+180, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(2 * SEP, paW+240, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(2 * SEP, paW+300, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),

    Pointing(*calculate_center(np.sqrt(3) * SEP, paW+30, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0, label='haloW_7'),
    Pointing(*calculate_center(np.sqrt(3) * SEP, paW+90, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0, label='haloW_4'),
    #Pointing(*calculate_center(np.sqrt(3) * SEP, paW+150, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(np.sqrt(3) * SEP, paW+210, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(np.sqrt(3) * SEP, paW+270, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(np.sqrt(3) * SEP, paW+330, raW-0.35*SEP/np.cos(np.deg2rad(decW))*np.sin(np.deg2rad(paW)), decW-0.35*SEP*np.cos(np.deg2rad(paW))), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),

    #Pointing(*calculate_center(3 * SEP, paW+0, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),
    Pointing(*calculate_center(3 * SEP, paW+60, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0, label='haloW_9'),
    #Pointing(*calculate_center(3 * SEP, paW+120, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(3 * SEP, paW+180, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(3 * SEP, paW+240, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(3 * SEP, paW+300, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),

    #Pointing(*calculate_center(np.sqrt(7) * SEP, paW+20, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),
    Pointing(*calculate_center(np.sqrt(7) * SEP, paW+40, raW + 0.1*SEP/np.cos(np.deg2rad(decW)), decW + 0.1*SEP), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0, label='haloW_10'),
    Pointing(*calculate_center(np.sqrt(7) * SEP, paW+80, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0, label='haloW_5'),
    Pointing(*calculate_center(np.sqrt(7) * SEP, paW+100, raW+0.1*SEP/np.cos(np.deg2rad(decW)), decW-0.3*SEP), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0, label='haloW_3'),
    #Pointing(*calculate_center(np.sqrt(7) * SEP, paW+140, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(np.sqrt(7) * SEP, paW+160, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(np.sqrt(7) * SEP, paW+200, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(np.sqrt(7) * SEP, paW+220, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(np.sqrt(7) * SEP, paW+260, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(np.sqrt(7) * SEP, paW+280, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(np.sqrt(7) * SEP, paW+320, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),
    #Pointing(*calculate_center(np.sqrt(7) * SEP, paW+340, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0),
    
    Pointing(*calculate_center(np.sqrt(13) * SEP, paW+46.102, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0, label='haloW_12'),
    Pointing(*calculate_center(np.sqrt(13) * SEP, paW+73.898, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0, label='haloW_8'),
    Pointing(*calculate_center(np.sqrt(13) * SEP, paW+106.102, raW+0.1*SEP/np.cos(np.deg2rad(decW)), decW-0.4*SEP), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0, label='haloW_2'),
    Pointing(*calculate_center(np.sqrt(13) * SEP, paW+106.102, raW+0.2*SEP/np.cos(np.deg2rad(decW)), decW+0.6*SEP), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0, label='haloW_1'),

    Pointing(*calculate_center(4 * SEP, paW+60, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0, label='haloW_11'),
    
    Pointing(*calculate_center(np.sqrt(21) * SEP, paW+49.107, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0, label='NWstream_1'),
    Pointing(*calculate_center(5.45 * SEP, paW+52, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0, label='NWstream_2'),
    Pointing(*calculate_center(5 * SEP, paW+60, raW, decW), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0, label='NWstream_3'),
    Pointing(*calculate_center(6 * SEP, paW+60, raW+0.42*SEP/np.cos(np.deg2rad(decW)), decW+0.35*SEP), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0, label='NWstream_4'),
    Pointing(*calculate_center(6 * SEP, paW+60, raW+0.6*SEP/np.cos(np.deg2rad(decW)), decW+1.35*SEP), posang=fieldpaW-paW, obs_time=obs_time, stage=0, priority=0, label='NWstream_5'),
]

# Plot the pointings

In [0]:
from astropy.coordinates import SkyCoord as sc
for i, p in enumerate(pointings):
    scp = sc(ra=p.ra, dec=p.dec, unit="deg")
    rastr = scp.ra.to_string(unit='hourangle', sep=':', precision=2, pad=2)
    decstr = scp.dec.to_string(sep=':', precision=1, pad=2, alwayssign=True)
    print(f"        ('sector_0', '{p.label}',   '{rastr}',  '{decstr}', {p.posang}, {p.stage}, 0),")

In [0]:
obs

In [0]:
cmap = plt.get_cmap('tab20')

f, ax = plt.subplots(1, 1, figsize=(8, 8), dpi=240, subplot_kw=dict(projection=wcs.wcs))

obs.plot(ax, fov, c='lightgrey', size=0.5, s=np.s_[::50])

# Plot the tidal radius from the King profile fit
#ell = king.get_ellipse(R_t)
#fov.plot(ax, ell, native_frame='world', ls='--', lw=1, c='k')
#ell = king.get_ellipse(R_c)
#fov.plot(ax, ell, native_frame='world', ls='-', lw=1, c='k')

# Plot the poiting center
# fov.plot(ax, [field.pos.ra.value], [field.pos.dec.value], fmt='+r', ms=10, native_frame='world')

for i, pp in enumerate(pointings):
    wfc = SubaruWFC(pp)
    pfi = SubaruPFI(wfc, instrument_options=config.instrument_options)

    pfi.plot_focal_plane(ax, fov, corners=True, c=cmap((pp.stage + 1) % 20), ls='-', label=pp.label)
    pfi.plot_focal_plane(ax, fov, fill=True, alpha=0.2, c=cmap((pp.stage + 1) % 20))

    if pp.label in ['diskE_1', 'diskE_2', 'NWstream_5', 'GSS_1', 'haloE_1', 'diskW_2', 'diskW_3', 'haloW_4']:
        color = 'red'
    else:
        color = 'black'

    ax.text(pp.ra, pp.dec,
            f'{pp.label}\n{pp.priority}',
            color=color, fontsize=8, ha='center', va='center', transform=ax.get_transform('world'))

ax.set_xlim(10, -10)
ax.set_ylim(-10, 10)
ax.grid()
#ax.legend()

# Calculate and plot the elevation and rotator angle

In [0]:
obs = Observer.at_site("Subaru", timezone="US/Hawaii")
hawaii_tz = TimeDelta(-10 * u.hr)

In [0]:
config.pointings

In [0]:
time_from = Time(dateutil.parser.parse(TIME_FROM)) - hawaii_tz
time_to = Time(dateutil.parser.parse(TIME_TO)) - hawaii_tz

time_from, time_to

In [0]:
date_from = np.floor(time_from.jd - 0.5)
date_to = np.ceil(time_to.jd + 0.5)

dates = Time(np.linspace(date_from, date_to + 1, int(date_to - date_from + 2)), format='jd')
dates.to_datetime()

In [0]:
sunrise = obs.sun_rise_time(dates, which='previous') + hawaii_tz
sunset = obs.sun_set_time(dates, which='next') + hawaii_tz

# HST
sunrise.to_datetime(), sunset.to_datetime()

In [0]:
def radecpa_to_altazinr(pos, pa, obs_time):
    altaz = pos.transform_to(AltAz(obstime=obs_time, location=Subaru_POPT2_PFS.Lsbr))
    az = altaz.az.deg
    alt = altaz.alt.deg

    # Instrument rotator angle
    subaru = Subaru_POPT2_PFS.Subaru()
    paa = subaru.radec2inr(pos.ra, pos.dec, obs_time)
    inr = (paa + pa + 180) % 360. - 180.
    # inr = paa + pa

    return alt, az, inr

In [0]:
obs_time = Time(np.linspace(date_from, date_to, 300), format='jd')
obs_time[0], obs_time[-1]

In [0]:
obs_time[0], obs_time[-1]

In [0]:
f, ax = plt.subplots(1, 1, figsize=(7, 5), dpi=240)
ax2 = ax.twinx()

# matplotlib like python datetime
xx = (obs_time + hawaii_tz).to_datetime()

# Shade areas in black when the Sun is below the horizon
for t1, t2 in zip(sunset[:-2].to_datetime(), sunrise[2:].to_datetime()):
    ax.axvspan(t1, t2, color='black', alpha=1)

# Plot the rotator angle for each pointing
for p in pointings:
    if p.label in ['diskE_1', 'diskE_2', 'NWstream_5', 'GSS_1', 'haloE_1', 'diskW_2', 'diskW_3', 'haloW_4']:
        #p = p.get_pointing()
        alt, az, inr = radecpa_to_altazinr(p.pos, p.posang, obs_time=(obs_time))
        # alt, az, inr = radecpa_to_altazinr(p.pos, 0, obs_time=(obs_time))

        inr_mask = np.abs(inr) > 178
        inr[inr_mask] = np.nan

        # Instrument rotator angle
        ax2.plot(xx, inr, c='orange')

        # Plot the azimuth angle
        # target_ha = obs.target_hour_angle(obs_time, target=p.pos)
        ax2.plot(xx, ((az + 180.) % 360) - 180., c='green')

# Plot the elevation of the Moon
moon = get_body('Moon', obs_time, Subaru_POPT2_PFS.Lsbr)
moon_phase = moon_illumination(obs_time)
altaz = moon.transform_to(AltAz(obstime=obs_time, location=Subaru_POPT2_PFS.Lsbr))
ax.plot(xx, altaz.alt.degree, '--y')

# Plot target elevation
p = field.get_center_pointing(include_motion=False)
alt, az, inr = radecpa_to_altazinr(p.pos, 0, obs_time=(obs_time))
ax.plot(xx, alt, '-c')

ax.set_xlim((time_from + hawaii_tz).to_datetime(), (time_to + hawaii_tz).to_datetime())

ax.set_yticks(np.linspace(0, 90, 7))
ax.set_ylim(0, 90)

ax.tick_params(axis='x', labelrotation=90)

ax.set_xlabel('time [HST]')
ax.set_ylabel('elevation')
ax.set_title(f'Field: {FIELD}, moon phase: {moon_phase.mean():0.2f}')

ax.grid(True)

ax2.set_yticks(np.linspace(-180, 180, 7))
ax2.set_ylim(-180, 180)

ax2.set_ylabel('rotator angle')

In [0]:
# Calculate the precise time of meridian crossing for each pointing

# Sample time at minute resolution
# Time constants are HST, t_fr and t_to are GMT
t_fr = Time('2025-04-01T18:00:00') - hawaii_tz
t_to = Time('2025-04-02T06:00:00') - hawaii_tz
td = TimeDelta(1 * u.min)
t_fr.jd, t_to.jd, td.jd

In [0]:
t = Time(np.arange(t_fr.jd, t_to.jd, td.jd), format='jd')
tt = (t + hawaii_tz).to_datetime()
t.shape

# Meridian crossing for each pointing in HST

In [0]:
for i, p in enumerate(config.pointings):
    p = p.get_pointing()
    alt, az, inr = radecpa_to_altazinr(p.pos, p.posang, obs_time=(t))

    # Wrap around az so it crossed 0 at the meridian
    az = ((az + 180.) % 360) - 180.

    # Find the zero crossing
    m = (-10 <= az) & (az <= 10)
    ip = interp1d(az[m], t[m].jd)
    t_m = ip(0)

    # Print the meridian-crossing in HST for each pointing
    print(i, p.ra, p.dec, p.posang, (Time(t_m, format='jd') + hawaii_tz).iso)

In [0]:
plt.plot(tt, az)
plt.plot(tt[m], az[m])
plt.axhline(0, c='r')
plt.grid()